# crop tank vids

Edit tank vides to remove sections without a fish. Create new videos, with one video per fish appearance.

In [1]:
from scipy.signal import medfilt
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
from pathlib import Path
import numpy as np
import subprocess
import pickle
import random
import time
import cv2
import os


# settings
root_dir = r'Z:\locker\ShareData\Chin_for_Dillon'
bg_frame_num = 500    # frames to include in background calculation
diff_dt = .1          # (seconds) time between background frame computation
thresh = 4.0;         # (z scored pixels) difference between frame and background frame for fish detection
buffer = 1.5;         # (s) time before and after fish appearance to keep
med_filtering = 1.0;  # (s) median filter the thresholded signal to remove short periods
bit_rate = 12;        # (megabits per second)


In [2]:
def crop_video(video, max_vids=20):

    # initialize video
    print(f'\n------- ANALYZING {video} -------')
    vid = cv2.VideoCapture(video)
    n_frames_approx = max(int(vid.get(cv2.CAP_PROP_FRAME_COUNT)), 10)  # approximate (assumes at least 10 frames)
    fps = int(vid.get(cv2.CAP_PROP_FPS))
    frame_delta = round(diff_dt * fps)
    name, ext = os.path.splitext(video)

    # compute background
    # (median across randomly sampled frames)
    bg_frame_nums = random.sample(range(n_frames_approx), min(n_frames_approx, bg_frame_num))
    bg_frame_nums.sort()

    bg_frames = []
    for i in bg_frame_nums:
        vid.set(1, i)  # using `set` instead of reading through all the frames is less accurate but much faster
        _, frame = vid.read();
        bg_frames.append(frame)
    vid.release()
    bg = np.median(np.array(bg_frames), axis=0)

    # compute differences between frames and background
    vid = cv2.VideoCapture(video)  # reinitialize
    got_frame = True
    diffs = []
    delta_counter = frame_delta-1  # only compute background every delta frames
    n_frames = 0

    while got_frame:
        got_frame, frame = vid.read();
        if got_frame: n_frames += 1
        delta_counter += 1
        if got_frame and delta_counter==frame_delta:
            diffs.append(np.abs(frame - bg).mean())
            delta_counter = 0
    vid.release()
    diffs = np.array(diffs)

    # find fish appearances
    t = np.linspace(0, diffs.shape[0]*(frame_delta/fps), len(diffs))
    threshed = diffs > thresh
    kernel_sz = round(med_filtering*fps/frame_delta)
    kernel_sz = kernel_sz - kernel_sz%2 + 1  # make odd
    threshed = medfilt(threshed, kernel_sz)  # median filtering to debounce

    # plot
    fig = plt.figure(figsize=(10,2.5))
    ax = plt.axes(xlabel='time (s)', ylabel='frame diffs')
    ax.plot(t, diffs);
    ax.plot(t, threshed*np.ptp(diffs) + min(diffs));
    plt.savefig(os.path.join(os.path.dirname(video), 'cropping.png'), facecolor=(1,1,1,1))
    plt.savefig(os.path.join(root_dir, 'cropping', '{}_{}_cropping.png'.format(
        os.path.basename(str(Path(video).parents[1])),
        os.path.basename(str(Path(video).parents[0])) )), facecolor=(1,1,1,1))
    plt.close()
    
    # create vids for each epoch
    if not threshed.any():
        print('fish not detected.')
        return

    start_inds = np.where(np.diff(threshed)==1)[0]+1
    end_inds   = np.where(np.diff(threshed)==-1)[0]+1
    if len(start_inds)==0 and end_inds:
        start_inds = [0]
    elif len(end_inds)==0 and start_inds:
        end_inds = [len(diffs)]
    if start_inds[0] < end_inds[0]:
        np.insert(start_inds, 0, 0)
    if start_inds[-1] > end_inds[-1]:
        end_inds.append(len(diffs))

    # create new cropped videos
    for i, (start_ind, end_ind) in enumerate(zip(start_inds[:max_vids], end_inds[:max_vids])):
        output_name = '{}_cropped{:03d}{}'.format(name, i, ext)
        start = max(0, start_ind*(frame_delta/fps) - buffer)
        end   = min(n_frames/fps, end_ind*(frame_delta/fps) + buffer)
        print('cropping from {:.1f} to {:.1f} seconds ({:03d})'.format(start, end, i))
        command = 'ffmpeg -i {} -ss {:.2f} -t {:.2f} -vb {}M -y -loglevel panic -stats {}'.format(
            video, start, end-start, bit_rate, output_name)
        os.system(command)


In [14]:
vid_folders = [f.path for f in os.scandir(root_dir) if f.is_dir()]  # or define manually
problem_folders = []

for vid_folder in vid_folders:
    sub_folders = [f.path for f in os.scandir(vid_folder) if f.is_dir()]
    for sub_folder in sub_folders:
        try:
            # delete old cropped videos if exist
            old_cropped_vids = [file for file in os.listdir(sub_folder) if 'cropped' in file and file.endswith('.avi')]
            for old_vid in old_cropped_vids:
                os.remove(os.path.join(sub_folder, old_vid))
            
            crop_video(os.path.join(sub_folder, 'videoEOD.avi'))
        except:
            print('PROBLEM ANALYZING {}'.format(os.path.join(sub_folder, 'videoEOD.avi')))
            problem_folders.append((vid_folder, sub_folder))



------- ANALYZING Z:\locker\ShareData\Chin_for_Dillon\20200903\time_h11m28s22\videoEOD.avi -------
cropping from 22.3 to 30.1 seconds (000)

------- ANALYZING Z:\locker\ShareData\Chin_for_Dillon\20200903\time_h11m30s47\videoEOD.avi -------
cropping from 14.6 to 20.1 seconds (000)

------- ANALYZING Z:\locker\ShareData\Chin_for_Dillon\20200903\time_h11m31s58\videoEOD.avi -------
cropping from 18.7 to 25.2 seconds (000)

------- ANALYZING Z:\locker\ShareData\Chin_for_Dillon\20200903\time_h11m32s31\videoEOD.avi -------
cropping from 23.0 to 29.7 seconds (000)

------- ANALYZING Z:\locker\ShareData\Chin_for_Dillon\20200903\time_h11m33s4\videoEOD.avi -------
cropping from 111.0 to 115.4 seconds (000)

------- ANALYZING Z:\locker\ShareData\Chin_for_Dillon\20200903\time_h11m35s11\videoEOD.avi -------
WARNING! Fish not detected!

------- ANALYZING Z:\locker\ShareData\Chin_for_Dillon\20200903\time_h11m38s9\videoEOD.avi -------
cropping from 37.9 to 42.4 seconds (000)
cropping from 51.0 to 59.5


------- ANALYZING Z:\locker\ShareData\Chin_for_Dillon\20200903\time_h14m12s31\videoEOD.avi -------
WARNING! Fish not detected!

------- ANALYZING Z:\locker\ShareData\Chin_for_Dillon\20200903\time_h14m12s59\videoEOD.avi -------
cropping from 31.7 to 37.4 seconds (000)

------- ANALYZING Z:\locker\ShareData\Chin_for_Dillon\20200903\time_h14m13s45\videoEOD.avi -------
cropping from 0.0 to 3.0 seconds (000)

------- ANALYZING Z:\locker\ShareData\Chin_for_Dillon\20200903\time_h14m14s12\videoEOD.avi -------
cropping from 3.2 to 10.2 seconds (000)

------- ANALYZING Z:\locker\ShareData\Chin_for_Dillon\20200903\time_h14m14s26\videoEOD.avi -------
cropping from 0.1 to 5.3 seconds (000)

------- ANALYZING Z:\locker\ShareData\Chin_for_Dillon\20200903\time_h14m14s39\videoEOD.avi -------
cropping from 5.3 to 9.5 seconds (000)

------- ANALYZING Z:\locker\ShareData\Chin_for_Dillon\20200903\time_h14m14s51\videoEOD.avi -------
WARNING! Fish not detected!

------- ANALYZING Z:\locker\ShareData\Chin_fo

D:\anaconda3\envs\dpk\lib\site-packages\ipykernel_launcher.py:66: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
D:\anaconda3\envs\dpk\lib\site-packages\ipykernel_launcher.py:68: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.


PROBLEM ANALYZING Z:\locker\ShareData\Chin_for_Dillon\20200903\time_h15m31s10\videoEOD.avi

------- ANALYZING Z:\locker\ShareData\Chin_for_Dillon\20200903\time_h15m31s53\videoEOD.avi -------
WARNING! Fish not detected!

------- ANALYZING Z:\locker\ShareData\Chin_for_Dillon\20200903\time_h15m32s10\videoEOD.avi -------
WARNING! Fish not detected!

------- ANALYZING Z:\locker\ShareData\Chin_for_Dillon\20200903\time_h15m32s41\videoEOD.avi -------
cropping from 21.3 to 26.1 seconds (000)
cropping from 58.1 to 65.0 seconds (001)

------- ANALYZING Z:\locker\ShareData\Chin_for_Dillon\20200903\time_h15m33s51\videoEOD.avi -------
WARNING! Fish not detected!

------- ANALYZING Z:\locker\ShareData\Chin_for_Dillon\20200903\time_h15m34s10\videoEOD.avi -------
cropping from 11.6 to 16.0 seconds (000)

------- ANALYZING Z:\locker\ShareData\Chin_for_Dillon\20200903\time_h15m34s29\videoEOD.avi -------
cropping from 25.9 to 29.8 seconds (000)

------- ANALYZING Z:\locker\ShareData\Chin_for_Dillon\202009

D:\anaconda3\envs\dpk\lib\site-packages\ipykernel_launcher.py:66: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
D:\anaconda3\envs\dpk\lib\site-packages\ipykernel_launcher.py:68: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.


...

D:\anaconda3\envs\dpk\lib\site-packages\ipykernel_launcher.py:66: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
D:\anaconda3\envs\dpk\lib\site-packages\ipykernel_launcher.py:68: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.


PROBLEM ANALYZING Z:\locker\ShareData\Chin_for_Dillon\Charuco\time_h11m16s6\videoEOD.avi

------- ANALYZING Z:\locker\ShareData\Chin_for_Dillon\Fish1\time_h12m10s49\videoEOD.avi -------
cropping from 63.9 to 7.3 seconds (000)
cropping from 130.1 to 68.7 seconds (001)
cropping from 206.5 to 135.2 seconds (002)
cropping from 301.1 to 211.0 seconds (003)
cropping from 354.9 to 306.2 seconds (004)
cropping from 554.3 to 360.0 seconds (005)
cropping from 649.8 to 558.7 seconds (006)
cropping from 748.5 to 658.4 seconds (007)
cropping from 770.0 to 753.7 seconds (008)
cropping from 853.3 to 774.4 seconds (009)

------- ANALYZING Z:\locker\ShareData\Chin_for_Dillon\Fish1\time_h12m1s31\videoEOD.avi -------
cropping from 1.0 to 12.0 seconds (000)
cropping from 21.2 to 27.3 seconds (001)

------- ANALYZING Z:\locker\ShareData\Chin_for_Dillon\Fish1\time_h12m28s22\videoEOD.avi -------
cropping from 0.0 to 9.5 seconds (000)
cropping from 34.0 to 39.3 seconds (001)

------- ANALYZING Z:\locker\Share

cropping from 5.3 to 12.6 seconds (000)

------- ANALYZING Z:\locker\ShareData\Chin_for_Dillon\Fish3\time_h14m47s51\videoEOD.avi -------
cropping from 6.8 to 11.8 seconds (000)
cropping from 31.3 to 36.2 seconds (001)

------- ANALYZING Z:\locker\ShareData\Chin_for_Dillon\Fish4\time_h15m10s55\videoEOD.avi -------
WARNING! Fish not detected!

------- ANALYZING Z:\locker\ShareData\Chin_for_Dillon\Fish4\time_h15m11s1\videoEOD.avi -------
cropping from 0.0 to 52.7 seconds (000)
cropping from 55.0 to 71.2 seconds (001)

------- ANALYZING Z:\locker\ShareData\Chin_for_Dillon\Fish4\time_h15m12s16\videoEOD.avi -------


D:\anaconda3\envs\dpk\lib\site-packages\ipykernel_launcher.py:66: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
D:\anaconda3\envs\dpk\lib\site-packages\ipykernel_launcher.py:68: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.


PROBLEM ANALYZING Z:\locker\ShareData\Chin_for_Dillon\Fish4\time_h15m12s16\videoEOD.avi

------- ANALYZING Z:\locker\ShareData\Chin_for_Dillon\Fish4\time_h15m12s28\videoEOD.avi -------
cropping from 6.9 to 22.5 seconds (000)
cropping from 21.1 to 32.0 seconds (001)
cropping from 36.7 to 45.3 seconds (002)

------- ANALYZING Z:\locker\ShareData\Chin_for_Dillon\Fish4\time_h15m14s2\videoEOD.avi -------
cropping from 6.5 to 19.0 seconds (000)
cropping from 17.1 to 25.0 seconds (001)

------- ANALYZING Z:\locker\ShareData\Chin_for_Dillon\Fish4\time_h15m15s14\videoEOD.avi -------
cropping from 12.9 to 2.4 seconds (000)

------- ANALYZING Z:\locker\ShareData\Chin_for_Dillon\Fish4\time_h15m15s43\videoEOD.avi -------
cropping from 0.0 to 9.3 seconds (000)

------- ANALYZING Z:\locker\ShareData\Chin_for_Dillon\Fish4\time_h15m16s16\videoEOD.avi -------
cropping from 0.0 to 7.0 seconds (000)

------- ANALYZING Z:\locker\ShareData\Chin_for_Dillon\Fish4\time_h15m16s4\videoEOD.avi -------
cropping fr

In [15]:
problem_folders

[('Z:\\locker\\ShareData\\Chin_for_Dillon\\20200903',
  'Z:\\locker\\ShareData\\Chin_for_Dillon\\20200903\\time_h15m31s10'),
 ('Z:\\locker\\ShareData\\Chin_for_Dillon\\20200903',
  'Z:\\locker\\ShareData\\Chin_for_Dillon\\20200903\\time_h15m47s57'),
 ('Z:\\locker\\ShareData\\Chin_for_Dillon\\20200911',
  'Z:\\locker\\ShareData\\Chin_for_Dillon\\20200911\\time_h17m39s3'),
 ('Z:\\locker\\ShareData\\Chin_for_Dillon\\20200911',
  'Z:\\locker\\ShareData\\Chin_for_Dillon\\20200911\\time_h17m53s54'),
 ('Z:\\locker\\ShareData\\Chin_for_Dillon\\Charuco',
  'Z:\\locker\\ShareData\\Chin_for_Dillon\\Charuco\\time_h11m16s6'),
 ('Z:\\locker\\ShareData\\Chin_for_Dillon\\Fish2',
  'Z:\\locker\\ShareData\\Chin_for_Dillon\\Fish2\\time_h13m31s4'),
 ('Z:\\locker\\ShareData\\Chin_for_Dillon\\Fish4',
  'Z:\\locker\\ShareData\\Chin_for_Dillon\\Fish4\\time_h15m12s16')]